
# RFM Analizi ile Müşteri Segmantasyonu

---------

## İş Problemi

Online ayakkabı mağazası olan Flo müşterilerini segmentlere göre ayırıp bu segmentlere göre pazarlama stratejilerini belirlemek istiyor. Buna yöneli olarak müşterilerinin davranışlarını tanımlanacak ve bu davranışlardaki öbeklenmelere göre gruplar oluşturulacaktır.


-----

In [268]:
# kütüphaneleri import edelim; 
import pandas as pd
import datetime as dt

# pandas görüntü ayarlarını yapalım;
#pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
#pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [290]:

# veri setini dahil edelim;
df_ = pd.read_csv(r"C:\Users\kkakt\Desktop\Flo\dataset\flo_data_20k.csv")
df = df_.copy()

df.head()

,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12
0,cc294636-19f0-11eb-8d74-000d3a38a36f,Android App,Offline,2020-10-30,2021-02-26,2021-02-21,2021-02-26,4.0,1.0,139.99,799.38,[KADIN]
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,Android App,Mobile,2017-02-08,2021-02-16,2021-02-16,2020-01-10,19.0,2.0,159.97,1853.58,"[ERKEK, COCUK, KADIN, AKTIFSPOR]"
2,69b69676-1a40-11ea-941b-000d3a38a36f,Android App,Android App,2019-11-27,2020-11-27,2020-11-27,2019-12-01,3.0,2.0,189.97,395.35,"[ERKEK, KADIN]"
3,1854e56c-491f-11eb-806e-000d3a38a36f,Android App,Android App,2021-01-06,2021-01-17,2021-01-17,2021-01-06,1.0,1.0,39.99,81.98,"[AKTIFCOCUK, COCUK]"
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,Desktop,Desktop,2019-08-03,2021-03-07,2021-03-07,2019-08-03,1.0,1.0,49.99,159.99,[AKTIFSPOR]


In [270]:
# gözlemlerin betimsel istatistiklerini inceleyelim;
df.describe().T

,count,mean,std,min,25%,50%,75%,max
order_num_total_ever_online,19945.0,3.110855,4.225647,1.00,1.00,2.00,4.00,200.00
order_num_total_ever_offline,19945.0,1.913913,2.062880,1.00,1.00,1.00,2.00,109.00
customer_value_total_ever_offline,19945.0,253.922597,301.532853,10.00,99.99,179.98,319.97,18119.14
customer_value_total_ever_online,19945.0,497.321690,832.601886,12.99,149.98,286.46,578.44,45220.13


In [271]:
# değişkenlerin içerdiği boş değerleri gözlemleyelim;
df.isnull().sum()

master_id                            0
order_channel                        0
last_order_channel                   0
first_order_date                     0
last_order_date                      0
last_order_date_online               0
last_order_date_offline              0
order_num_total_ever_online          0
order_num_total_ever_offline         0
customer_value_total_ever_offline    0
customer_value_total_ever_online     0
interested_in_categories_12          0
dtype: int64

In [272]:
# değişkenlerin tiplerini inceleyelim; 
df.dtypes

master_id                             object
order_channel                         object
last_order_channel                    object
first_order_date                      object
last_order_date                       object
last_order_date_online                object
last_order_date_offline               object
order_num_total_ever_online          float64
order_num_total_ever_offline         float64
customer_value_total_ever_offline    float64
customer_value_total_ever_online     float64
interested_in_categories_12           object
dtype: object

In [273]:
# müşterilerin toplam sipariş sayılarını bir değişkene atayalım;
df["total_order_num"] = df["order_num_total_ever_online"] + df["order_num_total_ever_offline"]

# müşterilerin toplam alışveriş harcamalarını bir değişkene atayalım
df["total_customer_value"] = df["customer_value_total_ever_offline"] + df["customer_value_total_ever_online"]

In [274]:
# tarihi ifade eden değişken tiplerini date'e çevirelim;

date_columns = df.columns[df.columns.str.contains("date")]
df[date_columns] = df[date_columns].apply(pd.to_datetime)
df.info

<bound method DataFrame.info of                                   master_id order_channel last_order_channel  \
0      cc294636-19f0-11eb-8d74-000d3a38a36f   Android App            Offline   
1      f431bd5a-ab7b-11e9-a2fc-000d3a38a36f   Android App             Mobile   
2      69b69676-1a40-11ea-941b-000d3a38a36f   Android App        Android App   
3      1854e56c-491f-11eb-806e-000d3a38a36f   Android App        Android App   
4      d6ea1074-f1f5-11e9-9346-000d3a38a36f       Desktop            Desktop   
...                                     ...           ...                ...   
19940  727e2b6e-ddd4-11e9-a848-000d3a38a36f   Android App            Offline   
19941  25cd53d4-61bf-11ea-8dd8-000d3a38a36f       Desktop            Desktop   
19942  8aea4c2a-d6fc-11e9-93bc-000d3a38a36f       Ios App            Ios App   
19943  e50bb46c-ff30-11e9-a5e8-000d3a38a36f   Android App        Android App   
19944  740998d2-b1f7-11e9-89fa-000d3a38a36f   Android App        Android App   

      f

In [275]:
# müşteri dağılımlarını inceleyelim; 
df.groupby("order_channel")[["total_order_num","total_customer_value"]].agg({"count","mean","sum"})

total_order_num                    total_customer_value  \
                        count      mean      sum                count   
order_channel                                                           
Android App              9495  5.504897  52269.0                 9495   
Desktop                  2735  3.992687  10920.0                 2735   
Ios App                  2833  5.418637  15351.0                 2833   
Mobile                   4882  4.440598  21679.0                 4882   

                                       
                     mean         sum  
order_channel                          
Android App    823.492655  7819062.76  
Desktop        588.782984  1610321.46  
Ios App        891.634285  2525999.93  
Mobile         620.275125  3028183.16

In [276]:
# en fazla kazancı getiren 10 müşteriyi sıralayalım;
df.sort_values("total_customer_value",ascending=False).head(10)

,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12,total_order_num,total_customer_value
11150,5d1c466a-9cfd-11e9-9897-000d3a38a36f,Android App,Desktop,2013-10-11,2021-04-30,2021-04-30,2020-12-24,200.0,2.0,684.97,45220.13,"[AKTIFCOCUK, ERKEK, COCUK, KADIN, AKTIFSPOR]",202.0,45905.10
4315,d5ef8058-a5c6-11e9-a2fc-000d3a38a36f,Android App,Android App,2018-08-06,2021-02-23,2021-02-23,2020-07-06,67.0,1.0,130.49,36687.80,"[AKTIFCOCUK, ERKEK, KADIN, AKTIFSPOR]",68.0,36818.29
7613,73fd19aa-9e37-11e9-9897-000d3a38a36f,Ios App,Offline,2014-01-14,2021-05-18,2021-01-30,2021-05-18,81.0,1.0,1263.76,32654.34,"[ERKEK, COCUK, KADIN, AKTIFSPOR]",82.0,33918.10
13880,7137a5c0-7aad-11ea-8f20-000d3a38a36f,Ios App,Offline,2021-03-01,2021-04-13,2021-03-18,2021-04-13,10.0,1.0,538.94,30688.47,"[ERKEK, KADIN, AKTIFSPOR]",11.0,31227.41
9055,47a642fe-975b-11eb-8c2a-000d3a38a36f,Android App,Offline,2021-04-07,2021-04-27,2021-04-07,2021-04-27,1.0,3.0,18119.14,2587.20,[AKTIFSPOR],4.0,20706.34
7330,a4d534a2-5b1b-11eb-8dbd-000d3a38a36f,Desktop,Desktop,2020-02-16,2021-04-30,2021-04-30,2020-12-18,66.0,4.0,843.68,17599.89,"[ERKEK, KADIN, AKTIFSPOR]",70.0,18443.57
8068,d696c654-2633-11ea-8e1c-000d3a38a36f,Ios App,Ios App,2017-05-10,2021-04-13,2021-04-13,2019-08-15,69.0,1.0,82.48,16836.09,"[AKTIFCOCUK, ERKEK, COCUK, KADIN, AKTIFSPOR]",70.0,16918.57
163,fef57ffa-aae6-11e9-a2fc-000d3a38a36f,Mobile,Desktop,2016-11-08,2021-05-12,2021-05-12,2020-07-09,36.0,1.0,180.73,12545.37,"[ERKEK, AKTIFSPOR]",37.0,12726.10
7223,cba59206-9dd1-11e9-9897-000d3a38a36f,Android App,Android App,2013-02-21,2021-05-09,2021-05-09,2020-01-25,130.0,1.0,49.99,12232.25,"[AKTIFCOCUK, ERKEK, KADIN, AKTIFSPOR]",131.0,12282.24
18767,fc0ce7a4-9d87-11e9-9897-000d3a38a36f,Desktop,Desktop,2018-11-24,2020-11-11,2020-11-11,2019-12-06,18.0,2.0,64.97,12038.18,"[ERKEK, KADIN]",20.0,12103.15


In [277]:
# en fazla sipariş veren 10 müşteriyi sıralyalım;
df.sort_values("total_order_num",ascending=False).head(10)

,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12,total_order_num,total_customer_value
11150,5d1c466a-9cfd-11e9-9897-000d3a38a36f,Android App,Desktop,2013-10-11,2021-04-30,2021-04-30,2020-12-24,200.0,2.0,684.97,45220.13,"[AKTIFCOCUK, ERKEK, COCUK, KADIN, AKTIFSPOR]",202.0,45905.10
7223,cba59206-9dd1-11e9-9897-000d3a38a36f,Android App,Android App,2013-02-21,2021-05-09,2021-05-09,2020-01-25,130.0,1.0,49.99,12232.25,"[AKTIFCOCUK, ERKEK, KADIN, AKTIFSPOR]",131.0,12282.24
8783,a57f4302-b1a8-11e9-89fa-000d3a38a36f,Android App,Offline,2019-08-07,2020-11-04,2020-09-07,2020-11-04,2.0,109.0,10239.46,143.98,"[AKTIFCOCUK, ERKEK, COCUK, KADIN, AKTIFSPOR]",111.0,10383.44
2619,fdbe8304-a7ab-11e9-a2fc-000d3a38a36f,Android App,Offline,2018-10-18,2020-06-30,2018-10-18,2020-06-30,1.0,87.0,8432.25,139.98,[ERKEK],88.0,8572.23
6322,329968c6-a0e2-11e9-a2fc-000d3a38a36f,Ios App,Ios App,2019-02-14,2021-04-05,2021-04-05,2020-02-17,2.0,81.0,3997.55,242.81,[ERKEK],83.0,4240.36
7613,73fd19aa-9e37-11e9-9897-000d3a38a36f,Ios App,Offline,2014-01-14,2021-05-18,2021-01-30,2021-05-18,81.0,1.0,1263.76,32654.34,"[ERKEK, COCUK, KADIN, AKTIFSPOR]",82.0,33918.10
9347,44d032ee-a0d4-11e9-a2fc-000d3a38a36f,Mobile,Mobile,2019-02-11,2021-02-11,2021-02-11,2020-12-24,3.0,74.0,4640.77,543.28,"[KADIN, AKTIFSPOR]",77.0,5184.05
10954,b27e241a-a901-11e9-a2fc-000d3a38a36f,Mobile,Mobile,2015-09-12,2021-04-01,2021-04-01,2019-10-08,72.0,3.0,292.93,5004.95,"[AKTIFCOCUK, ERKEK, KADIN, AKTIFSPOR]",75.0,5297.88
8068,d696c654-2633-11ea-8e1c-000d3a38a36f,Ios App,Ios App,2017-05-10,2021-04-13,2021-04-13,2019-08-15,69.0,1.0,82.48,16836.09,"[AKTIFCOCUK, ERKEK, COCUK, KADIN, AKTIFSPOR]",70.0,16918.57
7330,a4d534a2-5b1b-11eb-8dbd-000d3a38a36f,Desktop,Desktop,2020-02-16,2021-04-30,2021-04-30,2020-12-18,66.0,4.0,843.68,17599.89,"[ERKEK, KADIN, AKTIFSPOR]",70.0,18443.57


In [278]:
# ön hazırlık sürecinin fonskiyonlaştırılması

def data_prep(df):
    df["total_order_num"] = df["order_num_total_ever_online"] + df["order_num_total_ever_offline"]
    df["total_customer_value"] = df["customer_value_total_ever_offline"] + df["customer_value_total_ever_online"]
    date_columns = df.columns[df.columns.str.contains("date")]
    df[date_columns] = df[date_columns].apply(pd.to_datetime)
    return df

--------

## RFM Metriklerinin Oluşturulması

Rfm metrikleri recency,frequency,monetary değerlerinden oluşmaktadır.
* Recency: Müşteriyle son etkileşimde bulunduğumuz tarihtir.
* frequency: Müşterinin sipariş miktarıdır.
* monetary: Müşterinin bize bıraktığı hasılattır.

In [279]:
# Analiz günü belirleyim (max tarihten 2 gün sonra);
today_date = df["last_order_date"].max()
today_date = today_date + dt.timedelta(days=2)

In [281]:
rfm = pd.DataFrame()
# müşteri numarası hesaplayalım;
rfm["customer_id"] = df["master_id"]
# recency değerini hesaplayalım;
rfm["recency"] = (today_date - df["last_order_date"]).astype("timedelta64[D]")
# freqquency değerini hesaplayalım;
rfm["frequency"] = df["total_order_num"]
# monetary değerini hesaplayalım;
rfm["monetary"] = df["total_customer_value"]

rfm = rfm.set_index("customer_id")
rfm.head()

,recency,frequency,monetary
customer_id,,,
cc294636-19f0-11eb-8d74-000d3a38a36f,95.0,5.0,939.37
f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,105.0,21.0,2013.55
69b69676-1a40-11ea-941b-000d3a38a36f,186.0,5.0,585.32
1854e56c-491f-11eb-806e-000d3a38a36f,135.0,2.0,121.97
d6ea1074-f1f5-11e9-9346-000d3a38a36f,86.0,2.0,209.98


----------

## RF Skorunun Hesaplanması

In [282]:
# rfm metriklerini skora çevirelim;

rfm["recency_score"] = pd.qcut(rfm["recency"],5, labels = [5,4,3,2,1])
rfm["frequency_score"] = pd.qcut(rfm["frequency"].rank(method="first"),5,labels = [1,2,3,4,5])
rfm["monetary_score"] = pd.qcut(rfm["monetary"],5,labels = [1,2,3,4,5])
rfm.head()

,recency,frequency,monetary,recency_score,frequency_score,monetary_score
customer_id,,,,,,
cc294636-19f0-11eb-8d74-000d3a38a36f,95.0,5.0,939.37,3,4,4
f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,105.0,21.0,2013.55,3,5,5
69b69676-1a40-11ea-941b-000d3a38a36f,186.0,5.0,585.32,2,4,3
1854e56c-491f-11eb-806e-000d3a38a36f,135.0,2.0,121.97,3,1,1
d6ea1074-f1f5-11e9-9346-000d3a38a36f,86.0,2.0,209.98,3,1,1


In [283]:
# rf score oluşturalım;

rfm["RF_SCORE"] = rfm["recency_score"].astype("string") + rfm["frequency_score"].astype("string")
rfm.head()

,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RF_SCORE
customer_id,,,,,,,
cc294636-19f0-11eb-8d74-000d3a38a36f,95.0,5.0,939.37,3,4,4,34
f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,105.0,21.0,2013.55,3,5,5,35
69b69676-1a40-11ea-941b-000d3a38a36f,186.0,5.0,585.32,2,4,3,24
1854e56c-491f-11eb-806e-000d3a38a36f,135.0,2.0,121.97,3,1,1,31
d6ea1074-f1f5-11e9-9346-000d3a38a36f,86.0,2.0,209.98,3,1,1,31


-----

## RF Skorunun Segment Olarak Tanımlanması

In [284]:
# segmentlerin tanımlanması 
seg_map = {
        r'[1-2][1-2]': 'hibernating',
        r'[1-2][3-4]': 'at_risk',
        r'[1-2]5': 'cant_loose',
        r'3[1-2]': 'about_to_sleep',
        r'33': 'need_attention',
        r'[3-4][4-5]': 'loyal_customers',
        r'41': 'promising',
        r'51': 'new_customers',
        r'[4-5][2-3]': 'potential_loyalists',
        r'5[4-5]': 'champions'
    }

In [285]:
# rf skoruna göre segmentlerin replace yapılması

rfm["segment"] = rfm["RF_SCORE"].replace(seg_map,regex=True)
rfm.head()

,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RF_SCORE,segment
customer_id,,,,,,,,
cc294636-19f0-11eb-8d74-000d3a38a36f,95.0,5.0,939.37,3,4,4,34,loyal_customers
f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,105.0,21.0,2013.55,3,5,5,35,loyal_customers
69b69676-1a40-11ea-941b-000d3a38a36f,186.0,5.0,585.32,2,4,3,24,at_risk
1854e56c-491f-11eb-806e-000d3a38a36f,135.0,2.0,121.97,3,1,1,31,about_to_sleep
d6ea1074-f1f5-11e9-9346-000d3a38a36f,86.0,2.0,209.98,3,1,1,31,about_to_sleep


------

## Segmentasyon Analizi

In [286]:
# segmentlerin recency,frequency,monetary ortalamalarını inceleyelim;

rfm.groupby("segment")["recency","frequency","monetary"].agg({"count","mean"})

C:\Users\kkakt\AppData\Local\Temp\ipykernel_9572\1310913616.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  rfm.groupby("segment")["recency","frequency","monetary"].agg({"count","mean"})


recency             frequency            monetary  \
                      count        mean     count       mean    count   
segment                                                                 
about_to_sleep         1629  113.785144      1629   2.401473     1629   
at_risk                3131  241.606835      3131   4.472373     3131   
cant_loose             1200  235.444167      1200  10.698333     1200   
champions              1932   17.106625      1932   8.934265     1932   
hibernating            3604  247.949501      3604   2.394007     3604   
loyal_customers        3361   82.594763      3361   8.374591     3361   
need_attention          823  113.828676       823   3.727825      823   
new_customers           680   17.917647       680   2.000000      680   
potential_loyalists    2938   37.155888      2938   3.304289     2938   
promising               647   58.921175       647   2.000000      647   

                                  
                            mean  
segment                           
about_to_sleep        359.008963  
at_risk               646.610236  
cant_loose           1474.468217  
champions            1406.625109  
hibernating           366.267062  
loyal_customers      1216.818616  
need_attention        562.142965  
new_customers         339.955544  
potential_loyalists   533.184466  
promising             335.672705

## case 1:
FLO bünyesine yeni bir kadın ayakkabı markası dahil ediyor. Dahil ettiği markanın ürün fiyatları genel müşteri
tercihlerinin üstünde. Bu nedenle markanın tanıtımı ve ürün satışları için ilgilenecek profildeki müşterilerle özel olarak
iletişime geçmek isteniliyor. Sadık müşterilerinden(champions, loyal_customers) ve kadın kategorisinden alışveriş
yapan kişiler özel olarak iletişim kurulacak müşteriler. Bu müşterilerin id numaralarını csv dosyasına kaydediniz.

In [287]:
#case 1 

private_customers = rfm[(rfm["segment"] == "champions") | (rfm["segment"]=="loyal_customers")]
private_customers.head()

,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RF_SCORE,segment
customer_id,,,,,,,,
cc294636-19f0-11eb-8d74-000d3a38a36f,95.0,5.0,939.37,3,4,4,34,loyal_customers
f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,105.0,21.0,2013.55,3,5,5,35,loyal_customers
cfbda69e-5b4f-11ea-aca7-000d3a38a36f,86.0,5.0,1054.69,3,4,5,34,loyal_customers
c2e15af2-9eed-11e9-9897-000d3a38a36f,13.0,20.0,2861.60,5,5,5,55,champions
241f0ad0-afb5-11e9-9757-000d3a38a36f,2.0,24.0,4413.29,5,5,5,55,champions


In [288]:
# CSV'e aktarmak
private_customers.index.to_series().to_csv("private_customers_CustomerID.csv",index=False)

## case 2:
Erkek ve Çocuk ürünlerinde %40'a yakın indirim planlanmaktadır. Bu indirimle ilgili kategorilerle ilgilenen geçmişte
iyi müşteri olan ama uzun süredir alışveriş yapmayan kaybedilmemesi gereken müşteriler, uykuda olanlar ve yeni
gelen müşteriler özel olarak hedef alınmak isteniyor. Uygun profildeki müşterilerin id'lerini csv dosyasına kaydediniz.

In [289]:
#case 2: 

discount_customers = rfm.loc [rfm["segment"].isin(["about_to_sleep","new_customers","cant_loose"])]

# csv'e_aktarmak
discount_customers.index.to_series().to_csv("discount_customer_ID.csv",index=False)


-----------

## Sürecin fonskiyonlaştırılması

In [261]:
def create_rfm(df,csv=False):
    #veri ön işleme
    df.dropna(inplace=True)
    df["total_order"] = df["order_num_total_ever_online"] + df["order_num_total_ever_offline"] 
    df["total_price"] = df["customer_value_total_ever_online"] + df["customer_value_total_ever_offline"]
    
    #tarih değişkenini datetime çevirme
    date_columns = df.columns[df.columns.str.contains("date")]
    df[date_columns] = df[date_columns].apply(pd.to_datetime)
    #analiz tarihi oluşturma 
    today_date = df["last_order_date"].max()
    today_date = today_date + dt.timedelta(days=2)
    
    #metriklerin oluşturulması
    rfm = pd.DataFrame()
    
    rfm["recency"] = (today_date - df["last_order_date"]).astype("timedelta64[D]")
    rfm["frequency"] = df["total_order"]
    rfm["monetary"] = df["total_price"]
    rfm.set_index("master_id")
      
    #skorların belirlenmesi
    rfm["recency_score"] = pd.qcut(df["recency"],5,labels=[5,4,3,2,1])
    rfm["frequency_score"] = pd.qcut(df["frequency"].rank(method="first"),5,labels=[1,2,3,4,5])
    rfm["monetary_score"] = pd.qcut(df["monetary"],5,labels=[1,2,3,4,5])
    #rf score belirlenemesi
    rfm["RF_SCORE"] = rfm["recency_score"].astype("string") + rfm["frequency_score"].astype("string") 
    
    #segmentlerin belirlenmesi
    seg_map = {
        r'[1-2][1-2]': 'hibernating',
        r'[1-2][3-4]': 'at_risk',
        r'[1-2]5': 'cant_loose',
        r'3[1-2]': 'about_to_sleep',
        r'33': 'need_attention',
        r'[3-4][4-5]': 'loyal_customers',
        r'41': 'promising',
        r'51': 'new_customers',
        r'[4-5][2-3]': 'potential_loyalists',
        r'5[4-5]': 'champions'
    }
    
    rfm["segment"] = rfm["RF_SCORE"].replace(seg_map,regex=True)
    
    if csv:
        rfm.to_csv("customers_segmentation_with_rfm.csv")
    return rfm